In [ ]:
import pandas as pd

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [ ]:
path = '/Users/xyuan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S2to5combined_20ul_40min.xlsx'
df_mock = load_data(path)
path = '/Users/xyuan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S6to10combined_20ul_40min.xlsx'
df_rsv = load_data(path)
df_mock.shape, df_rsv.shape

In [ ]:
df_24k = df_mock[(df_mock.Mass>24000)&(df_mock.Mass<25000)]
plotly_zone(df_24k, y='Vol')

In [ ]:
fpath = '~/Studio/tools/NGS_24129_5p.xlsx'
df_ngs_5p = pd.read_excel(fpath)
fpath = '~/Studio/tools/NGS_24129_3p.xlsx'
df_ngs_3p = pd.read_excel(fpath)

In [ ]:
df_sample = df_mock.copy()
df_sample.shape

In [ ]:
# df_ngs = df_ngs_5p.copy()
df_ngs = df_ngs_3p.copy()
df_ngs.head()

In [ ]:
import multiprocessing
from collections import namedtuple
ShiftHit = namedtuple('ShiftHit', 'shift hit')

def func(df_s1, df_s2, shift):
    dfm = match_dfs(df_s1, df_s2, shift=shift)
    sh = ShiftHit(shift=shift, hit=dfm.shape[0])
    return sh

PROCESSES = 8
params = [(df_ngs, df_sample, shift) for shift in np.arange(0, 160, 1)]
with multiprocessing.Pool(PROCESSES) as pool:
    shift_hits = pool.starmap(func, params)

df_shift_hits = pd.DataFrame(shift_hits)

In [ ]:
fig = px.scatter(df_shift_hits[df_shift_hits.hit<df_shift_hits.shape[0]], x='shift', y='hit', labels={'shift': 'Mass Shift', 'hit': 'Hit Count'})
fig.show()
# df_shift_hits

In [ ]:
df_sh_top = df_shift_hits[df_shift_hits.hit>1]
# df_sh_top = df_shift_hits.sort_values('hit', ascending=False).iloc[:10]
shifts = df_sh_top['shift']
df_sh_top.shape

In [ ]:
# find out data points which can generate base calling
def func(df_s1, df_s2, shift):
    _, df = peer_dfs(df_s1, df_s2, shift=shift)
    
    bcr = base_calling_random(df, base_only=True)
    df = bcr[0].copy()
    df['Shift'] = shift
    return df

PROCESSES = 8
params = [(df_ngs, df_sample, shift) for shift in shifts]
with multiprocessing.Pool(PROCESSES) as pool:
    df_basecallings = pool.starmap(func, params)

df_hits = pd.concat(df_basecallings).drop_duplicates()

In [ ]:
plotly_zone(df_hits, y='Shift')

In [ ]:
# df_hits_5p = df_hits.copy()
# shift_5p = df_hits.Shift.unique()
# shift_5p
df_hits_3p = df_hits.copy()
shift_3p = df_hits.Shift.unique()
shift_3p

In [ ]:
full_mass = 24203.24 #24217.27 #24189.23 #  #
pairs = list()
for i in shift_5p:
    for j in shift_3p:
        if i+j == 88:
            dft = df_hits_5p[df_hits_5p.Shift == i]
            mass_5p = dft.sort_values('Mass')['Mass']
            dft = df_hits_3p[df_hits_3p.Shift == j]
            mass_3p = dft.sort_values('Mass')['Mass']
            max_5p, max_3p = mass_5p.max(), mass_3p.max()
            min_5p, min_3p = mass_5p.min(), mass_3p.min()
            if min_5p + min_3p > full_mass or max_5p + max_3p < full_mass:
                continue
            pairs.append((i, j))
            print(i, j, '\n', mass_5p.to_list(), '\n', mass_3p.to_list())

In [ ]:
# pairs_24203 = pairs.copy()
# pairs_24189 = pairs.copy()
# pairs_24217 = pairs.copy()

In [ ]:
selected_5p = [i[0] for i in pairs]
selected_3p = [i[1] for i in pairs]
# selected_5p.extend([0, 14])
# selected_3p.extend([0, 14])
# selected_5p, selected_3p
pairs

In [ ]:
target_shifts = selected_5p.copy()
dft = df_hits_5p[df_hits_5p.Shift.isin(target_shifts)]
plotly_zones(df_hits_5p, dft, y='Shift')
target_shifts = selected_3p.copy()
dft = df_hits_3p[df_hits_3p.Shift.isin(target_shifts)]
plotly_zones(df_hits_3p, dft, y='Shift')
# bcr = base_calling_random(dft)
# plotly_basecalling(*bcr)

In [ ]:
# plotly_zones(df_mock, dft)
plotly_zone(dft, y='Shift')

bcr = base_calling_random(dft)
plotly_basecalling(*bcr)